## Setup

In [51]:
#imports
import os, sys, json, datetime, re # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import ast
import requests
import urllib3
import time
import glob
from tqdm import tqdm
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
# from pandas.core.common import SettingWithCopyWarning


from IPython.display import display_markdown


## Data Gathering

In [12]:
#get most recent file
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\quorum\bill data downloads')
files = glob.glob('*.xlsx')

if len(files) > 1:
    max_mtime = 0
    for file in files:
        mtime = os.stat(file).st_mtime
        if mtime > max_mtime:
            max_mtime = mtime
            max_file = file

    bill_file = max_file

else:
    bill_file = files[0]



In [14]:
#df creation
all_bills = pd.read_excel(bill_file)
all_bills.columns = ['bill', 'state','bill_label','title', 'bill_sum_ai', 'bill_sum','sponsors','status','last_action','source_link']

# %% early ed bills
# ec_subs =  [
#     "Child Care Comn",
#     "Day Care",
#     "Early Childhood Education",
#     "Newborns & Infants",
#     "Partnership For Children",
#     "Social Services"
# ]



In [91]:
#looking through and testing various titles and descriptions
# ed_bills = all_bills.loc[all_bills['title'].notna() & all_bills['title'].str.contains(r'.*[Ee]ducation.*|.*[Cc]hild.*|.*[Ss]chool.*|.*[Cc]harter.*|.*[Mm]ath.*|.*[Rr]ead.*|.*[Tt]each.*|.*[Pp]arent.*|.*[Kk]id.*|.*[Ss]tudent.*|.*[Cc]ollege.*|.*[Uu]niversit.*|.*[Tt]uition.*', regex=True)]
# print(len(all_bills))
# print(len(ed_bills))



# ed_bills = all_bills.loc[(all_bills['title'].notna() & (all_bills['title'].str.contains(r'[Ii]ccb-?[Oo]ce[Ee]duc-.?|[Ee]arly [Cc]h(i)?ld|^Sch\s?|[Hg]ighr [Ee]d|[Hh]igher ([Ee]d(ucation)?)?\s?([Ll]earning)?|.*[Ee]ducation.*|.*[Cc]hild.*|.*[Ss]chool.*|.*[Cc]harter.*|.*[Mm]ath.*|.*[Rr]ead.*|.*[Tt]each.*|.*[Pp]arent.*|.*[Kk]id.*|.*[Ss]tudent.*|.*[Cc]ollege.*|.*[Uu]niversit.*|.*[Tt]uition.*', regex=True, na=False))) | all_bills['bill_sum'].str.contains(r'[Ee]arly\s?-?[Cc]hildhood|[Ee]ducational\s?[Rr]equirements|[Pp]ost-?\s?[Ss]econdary\s?[Ee]ducation|[Cc]hild\s?[Ee]mploy', regex=True, na = True)]


ed_bills = all_bills.loc[
    (all_bills['title'].notna() & 
     (all_bills['title'].str.contains(
         r'[Ii]ccb-?[Oo]ce[Ee]duc-.?|[Ee]arly [Cc]h(i)?ld|^Sch\s?|[Hg]ighr [Ee]d|[Hh]igher ([Ee]d(ucation)?)?\s?([Ll]earning)?|.*[Ee]ducation.*|.*[Cc]hild.*|.*[Ss]chool.*|.*[Cc]harter.*|.*[Mm]ath.*|.*[Rr]ead.*|.*[Tt]each.*|.*[Pp]arent.*|.*[Kk]id.*|.*[Ss]tudent.*|.*[Cc]ollege.*|.*[Uu]niversit.*|.*[Tt]uition.*', 
         regex=True, na=False
     ))
    ) 
    | 
    all_bills['bill_sum'].str.contains(
        r'[Ee]arly\s?-?[Cc]hildhood|[Ee]ducational\s?[Rr]equirements|[Pp]ost-?\s?[Ss]econdary\s?[Ee]ducation|[Cc]hild\s?[Ee]mploy', 
        regex=True, na=True
    )
]

ed_bills.to_excel(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\quorum\bill data downloads\exports\ed_bills.xlsx', index = False)

C:\Users\clutz\AppData\Local\Temp\ipykernel_10096\1095051003.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (all_bills['title'].str.contains(


In [ ]:

ed_bills.loc[:,"sponsors_helper"] = np.nan
for i,j in enumerate(ed_bills['sponsors']):
    
    # print(str(j))
    

    if isinstance(j,float):
        continue
    sponsors = str(j).split(',')
    s_list = []
    for s in sponsors:
        # display_markdown(f'{s}', raw=True)
        try:
            lname_identifier = re.findall(r'\w+\s\([RD]-[A-Z]{2}-\d+\)', str(s))[0]
        except:
            continue
            # print('no last name')
            # print(str(s))
            # # print(type(j))
        
        
        house_or_senate = re.findall(r'[Ss]en\.|[Rr]ep\.|[Dd]el\.',str(s))
        # print(house_or_senate)                

        if len(house_or_senate) == 0:
            if re.search(r'[Ss]peaker', str(s)):
                chamber = 'House'
            else:
                print('not speaker but something else')
                print(s)

        else:
            if house_or_senate[0] == "Sen.":
                chamber = "Senate"
                # print("found a senate")
            elif house_or_senate[0] == "Rep." or house_or_senate[0] == "Del.":
                chamber = "House"

        # print(chamber)
        state = re.findall(r'-[A-Z]{2}-',str(lname_identifier))[0]
        state = state.strip('-')
        # print(lname_identifier)
        # print(str(state))
        
        district = re.findall(r'-\d+', str(lname_identifier))[0]
        district = district.lstrip('-').strip().lstrip('0')
        name = re.findall(r'^\w+', str(lname_identifier))[0]
        # print(str(district))
        if 'ND' in str(state):
            helper = state +'-'+ chamber + "-" + district +'-' + name
        elif "CT" in str(state) and chamber == "Senate":
            helper = state +'-'+ chamber + "-S" + district

        else:
            helper = state +'-'+ chamber + "-" + district
        s_list.append(helper)

    sponsor_ext = "|".join(s_list)
    # try:
    #     print(sponsor_ext)
    # except:
    #     display_markdown(f'{s}', raw=True)


    ed_bills.loc[i, 'sponsors_helper'] = str(sponsor_ext)


    

In [97]:
first_column = ed_bills.pop('sponsors_helper') 
ed_bills.insert(0, 'sponsors_helper', first_column) 
ed_bills.reset_index(inplace=True, drop=True)



In [ ]:
#This is used to check if the values match the compiled_info csv

helper_values = []
for i,j in enumerate(ed_bills['sponsors_helper']):
    helper_ls = str(j).split('|')
    helper_values.extend(helper_ls)



helper_values = list(set(helper_values))
df = pd.DataFrame({"helper":helper_values})
df.to_excel(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\quorum\bill data downloads\exports\unique_helpers.xlsx', index=False)

In [ ]:
#taking bill data and providing cumulative totals
sponsors_dfs = []
for i,j in enumerate(ed_bills['sponsors_helper']):
    sponsor_list = []
    bill_label_list = []
    
    sponsors = str(j).split('|')
    bill_number = ed_bills.loc[i,'bill_label']
    n = len(sponsors)
    print(bill_number)

    sponsor_list.extend(sponsors)
    bill_label_list.extend([str(bill_number)]*n)

    df = pd.DataFrame({'sponsors':sponsor_list, "bill_labels": bill_label_list})
    print(df.to_string())
    sponsors_dfs.append(df)

bills_and_sponsors = pd.concat(sponsors_dfs)
bills_and_sponsors.reset_index(inplace=True, drop=True)

In [ ]:
#grouping data together and getting list of events per legislator
grouped_df = bills_and_sponsors.groupby('sponsors')#['bill_labels']#.apply('|'.join).reset_index()


# agg({
#     'bills': lambda x: 
#         f"{sc} ({ac})" if not pd.isna(ac) else f"{sc}"
#         for sc, ac in zip(thi_states_df.loc[x.index, 'event name'], thi_states_df.loc[x.index, 'role'])),

# }).reset_index()
# # grouped_df.reset_index()
# grouped_df.rename(columns={'event name': 'events'}, inplace=True)

: 

In [ ]:

bills_clean_up = non_ed_bills.loc[non_ed_bills['bill_sum'].notna() & (non_ed_bills['bill_sum'].str.contains(r'[Hh]igher ([Ee]d(ucation)?)?\s?([Ll]earning)?'))]

# for bill in non_ed_bills['title']:
#     print('##############')
#     print(bill)


In [ ]:

# Iterate through each row in the 'bill_sum' column
for text in all_bills['title'].dropna():
    # Find all matches with 5 words of context before and after
    matches = re.findall(r'(?:\b\w+\b\s){0,10}\b[Ee]ducation\b(?:\s\b\w+\b){0,10}', text)
    matches_v2 = re.findall(r'.*[Ee]ducation.*|.*[Cc]hild.*|.*[Ss]chool.*|.*[Cc]harter.*|.*[Mm]ath.*|.*[Rr]ead.*|.*[Tt]each.*|.*[Pp]arent.*|.*[Kk]id.*|.*[Ss]tudent.*|.*[Cc]ollege.*|.*[Uu]niversit.*|.*[Tt]uition.*', str(text))


    
    # Print matches if found
    print('####################')
    for match in matches_v2:
        
        print("****")
        print(str(match))
        print('\n')






In [ ]:
ec_subs = ['child\s{0,1}care', 'early\s{0,1}childhood', 'preschool']
ec_pat = "|".join(ec_subs)
print(ec_pat)

%%

In [ ]:
print(all_bills.columns)
for bill in all_bills['bill_sum']:
    if re.search(f'{ec_pat}', str(bill).lower()):
        matches = re.findall(ec_pat, str(bill).lower())
        print('############################')
        print('***********')
        print(*matches)
        print('***********')
        print(bill)
        print('\n')
# %%

In [ ]:
ec_ed_bills = all_bills[all_bills.subjects.str.contains(ec_pat, regex = True, case=False)]

In [ ]:
ec_ed_bills.reset_index(inplace=True, drop=True)

% Higher Ed

In [ ]:
he_keywords = [
    "post-secondary transition",
    "equity gaps",
    "college-going rates",
    "workforce readiness",
    "certificate programs",
    "wraparound services",
    "stackable credentials",
    "student persistence",
    "retention strategies",
    "lifelong learning",
    "postsecondary barriers",
    "alternative pathways",
    "higher education institutions",
    "higher education",
    "legislative support for education",
    "student success metrics",
    "high quality credential",
    "vocational training",
    "attainment",
    "persistence",
    "resistance",
    "graduation"
]

In [ ]:
he_pat = r'\b(' + '|'.join(map(re.escape, he_keywords)) + r')\b'

he_pat = "|".join(he_keywords)

In [ ]:
print(he_pat)

In [ ]:
all_bills['bill_sum'].fillna('nan',inplace=True)

In [ ]:
he_ed_bills = all_bills[all_bills.bill_sum.str.contains(he_pat, regex = True, case=False)]
he_ed_bills.reset_index(inplace=True, drop=True)

In [ ]:
os.chdir(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Higher Ed')
he_ed_bills.to_csv('higher_ed_bills.csv', index=False)
# %%
# print(all_bills.columns)
for i,bill in enumerate(he_ed_bills['bill_sum']):
    # matches = re.findall(he_pat, str(bill).lower())
    matches = re.findall(r'graduation', str(bill).lower())
    if matches:
        grad_match = re.findall(r'higher ed|post-{0,1}secondary', str(bill).lower())
        if grad_match:
            continue
        
        else:
            print('############################')
            print(he_ed_bills.loc[i,'state'])
            print(he_ed_bills.loc[i,'bill'])
            print('############################')
            print('***********')
            print(*grad_match)
            print('***********')
            print(bill)
            print('\n')
    
        print('############################')
        print(he_ed_bills.loc[i,'bill'])
        print('############################')
        print('***********')
        print(*matches)
        print('***********')
        print(bill)
        print('\n')
# %%